In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import torch

In [2]:
from src.models import TranAD, iTransformer

In [3]:
plt.style.use([hep.style.ROOT, hep.style.firamath])

In [4]:
# configs for local results (sometimes reduced data sets)
config = {
 'iTransformer1': {'window': 10, 'steps': 1, 'latent': 2, 'eps': 5, 'lab': 'iTransformer: w10 s1 l2'},
 'iTransformer2': {'window': 100, 'steps': 50, 'latent': 2, 'eps': 5, 'lab': 'iTransformer: w100 s50 l2'},
 'iTransformer3': {'window': 10, 'steps': 5, 'latent': 2, 'eps': 5, 'lab': 'iTransformer: w10 s5 l2'},
 'iTransformer4': {'window': 100, 'steps': 10, 'latent': 2, 'eps': 5, 'lab': 'iTransformer w100 s10 l2'},
 'TranAD': {'window': 10, 'steps': 1, 'eps': 5, 'latent': '', 'lab': 'TranAD: w10 s1'},
 'MAD_GAN': {'lab': 'MAD_GAN'},
 'OmniAnomaly': {'lab': 'OmniAnomaly'},
 'LSTM_AE': {'lab': 'LSTM_AE'},
 'DAGMM': {'lab': 'DAGMM'},
 'USAD': {'lab': 'USAD'},
 'IF': {'lab': 'IF'},
 'None': {'lab': 'None'}
}

In [5]:
# # configs for lxplus results
# config = {
#  'iTransformer1': {'window': 10, 'steps': 1, 'latent': 2, 'eps': 100, 'lab': 'iTransformer: w10 s1 l2'},
#  'iTransformer2': {'window': 100, 'steps': 50, 'latent': 2, 'eps': 100, 'lab': 'iTransformer: w100 s50 l2'},
#  'iTransformer3': {'window': 10, 'steps': 5, 'latent': 2, 'eps': 100, 'lab': 'iTransformer: w10 s5 l2'},
#  'iTransformer4': {'window': 100, 'steps': 10, 'latent': 2, 'eps': 100, 'lab': 'iTransformer w100 s10 l2'},
#  'TranAD': {'window': 10, 'steps': 1, 'eps': 100, 'latent': '', 'lab': 'TranAD: w10 s1'},
# #  'MAD_GAN': {'lab': 'MAD_GAN'},
# #  'OmniAnomaly': {'lab': 'OmniAnomaly'},
# #  'LSTM_AE': {'lab': 'LSTM_AE'},
# #  'DAGMM': {'lab': 'DAGMM'},
# #  'USAD': {'lab': 'USAD'},
# #  'IF': {'lab': 'IF'},
# #  'None': {'lab': 'None'}
# }

In [ ]:
datasets = ['IEEECIS_new2.2', 'SMAP_new', 'MSL_new', 'UCR', 'SMD', 'SWaT_1D', 'GECCO', 'SMD']  #, 'ATLAS_TS']
models = ['iTransformer1', 'iTransformer2', 'TranAD']  # 'iTransformer3', 'iTransformer4', 'TranAD']
        #   'OmniAnomaly', 'MAD_GAN', 'LSTM_AE', 'DAGMM', 'USAD', 'IF', 'None'] 

all_paths = []
results_mean_std = {}
for dataset in datasets:
    for model in models:
        if 'iTransformer' in model and model != 'iTransformer':
            paths = glob.glob(f'iTransformer_reduced_data/iTransformer_{dataset}')
            # paths = glob.glob(f'iTransformer_results_lxplus/iTransformer_{dataset}')
        else:
            paths = glob.glob(f'{model}_reduced_data/{model}_{dataset}')
            # paths = glob.glob(f'{model}_results_lxplus/{model}_{dataset}')
        all_paths.extend(paths)
        if not paths:
            print(f'No paths found for {model} on {dataset}')
        feats = 30 if dataset == 'IEEECIS_new2.2' else -1

        for path in paths:
            if model in config.keys():
                if model == 'TranAD':
                    res_path = glob.glob(f"{path}/*n_window{config[model]['window']}_steps{config[model]['steps']}*feats{feats}*/results/res.csv")
                else:
                    res_path = glob.glob(f"{path}/*n_window{config[model]['window']}_steps{config[model]['steps']}_feats{feats}_eps{config[model]['eps']}_latent{config[model]['latent']}*/results/res.csv")
            elif model == 'None':
                res_path = glob.glob(f'{path}/*feats{feats}*/results/res.csv')
            elif model == 'IF':
                res_path = glob.glob(f'{path}/*feats{feats}*/results/res.csv')
            else:
                res_path = glob.glob(f'{path}/*n_window10_steps1*feats{feats}*/results/res.csv')

            if res_path:
                res_path = np.sort(res_path)
                print(model, dataset, len(res_path))
                # print(res_path)
                tmp = pd.DataFrame()
                for p in res_path:
                    res = pd.read_csv(p)
                    tmp = pd.concat((tmp, res.iloc[-3:]))

                key = path.split('/')[1]
                if 'iTransformer' in model and model != 'iTransformer':
                    idx = len('iTransformer')  # len(model) - 1
                    diff = len(model) - len('iTransformer')
                    # insert a number in the key to distinguish between the models at position idx
                    key = key[:idx] + model[-diff:] + key[idx:]

                mean_values = tmp.groupby('Unnamed: 0').mean()
                std_values = tmp.groupby('Unnamed: 0').std()
                mean_values = mean_values.reindex(['local_all', 'local_all_maj', 'global'])
                std_values = std_values.reindex(['local_all', 'local_all_maj', 'global'])
                results_mean_std[key] = {'mean': mean_values, 'std': std_values}
                # print(results_mean_std[key])
            else:
                print(f'No results found for {model} on {dataset}')

            break

# print(len(all_paths))
print(results_mean_std.keys())

In [15]:
plt.rcParams.update({'lines.markersize': 6})
plt.rcParams.update({'errorbar.capsize': 8})
plt.rcParams.update({'lines.linewidth': 2})

In [16]:
modes = ['local (incl. OR)', 'local (maj. voting)', 'global']

In [17]:
def plot_scores_mean_std_err(results, modes, datasets, models, metric='MCC', name=None, labels=None):
    # colors = plt.cm.plasma(np.linspace(0, 1, len(models)+1))
    fig, axs = plt.subplots(3, 1, figsize=(22, 16), sharex=True, sharey=True)

    for i, mode in enumerate(modes):
        for j, model in enumerate(models):
            scores = {'mean': np.empty(0), 'std': np.empty(0)}
            for dataset in datasets:
                key = f'{model}_{dataset}'
               
                for val in ['mean', 'std']:
                    if key in results:
                        scores[val] = np.append(scores[val], results[key][val][metric].iloc[i])
                    else:
                        scores[val] = np.append(scores[val], 0)
            
            x_positions = np.arange(len(datasets)) + j * 0.1  # add offset for each model
            if labels:
                axs[i].errorbar(x_positions, scores['mean'], yerr=scores['std'], fmt='o', label=labels[j])  #, color=colors[j % len(colors)], capsize=5)
            else:
                axs[i].errorbar(x_positions, scores['mean'], yerr=scores['std'], fmt='o', label=model)  #, color=colors[j % len(colors)], capsize=5)
            
            axs[i].set_xticks(np.arange(len(datasets)) + 0.1 * (len(models) - 1) / 2)
            axs[i].set_xticklabels(labels=datasets)
            if metric == 'MCC':
                axs[i].set_ylim(-1, 1)
            else:
                axs[i].set_ylim(top=1.0)
            if metric == 'ROC/AUC':
                axs[i].set_ylabel('ROC AUC')
            elif metric == 'f1':
                axs[i].set_ylabel('F1')
            else:
                axs[i].set_ylabel(metric)
            axs[i].set_title(mode)
        axs[i].legend(bbox_to_anchor=(1.02, 1), loc='upper left')

    plt.tight_layout()
    if name:
        if metric == 'ROC/AUC':
            metric = 'rocauc'
        plt.savefig(f'./studies_results_lxplus/{name}_{metric}.png', facecolor='w')
    plt.show()


In [18]:
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

In [19]:
def plot_scores_mean_std_err2(results, modes, datasets, models, metric='MCC', name=None, labels=None):
    num_datasets = len(datasets)
    num_subplots = (num_datasets + 2) // 3  # Calculate the number of subplots needed
    fig, axs = plt.subplots(num_subplots, 1, figsize=(22, 8 * num_subplots), sharey=True)
    if num_subplots == 1:
        axs = [axs]  # Make axs iterable if it's a single subplot
    colors = plt.cm.plasma(np.linspace(0, 1, len(models)+1))
    markers = ['o', 's', 'D']  # Different markers for different modes
    plt.rcParams.update({'lines.markersize': 8})

    width = 0.05  # the width of the bars
    x = np.arange(num_datasets)  # the label locations

    for j, model in enumerate(models):
        for i, mode in enumerate(modes):
            scores = {'mean': np.empty(0), 'std': np.empty(0)}
            for dataset in datasets:
                key = f'{model}_{dataset}'
                for val in ['mean', 'std']:
                    if key in results:
                        scores[val] = np.append(scores[val], results[key][val][metric].iloc[i])
                    else:
                        scores[val] = np.append(scores[val], 0)

            x_positions = x + (j * len(modes) + i) * width  # add offset for each mode and model
            if labels:
                for k in range(num_subplots):
                    start_idx = k * 3
                    end_idx = min((k + 1) * 3, num_datasets)
                    axs[k].errorbar(x_positions[start_idx:end_idx], scores['mean'][start_idx:end_idx], yerr=scores['std'][start_idx:end_idx], fmt=markers[i], label=f'{labels[j]} ({mode})', color=colors[j])
            else:
                for k in range(num_subplots):
                    start_idx = k * 3
                    end_idx = min((k + 1) * 3, num_datasets)
                    axs[k].errorbar(x_positions[start_idx:end_idx], scores['mean'][start_idx:end_idx], yerr=scores['std'][start_idx:end_idx], fmt=markers[i], label=f'{model} ({mode})', color=colors[j])

    for k in range(num_subplots):
        start_idx = k * 3
        end_idx = min((k + 1) * 3, num_datasets)
        axs[k].set_xticks(x[start_idx:end_idx] + (len(modes) * len(models) - 1) * width / 2)
        axs[k].set_xticklabels(labels=datasets[start_idx:end_idx])
        if metric == 'MCC':
            axs[k].set_ylim(-0.2, 1.05)
        else:
            axs[k].set_ylim(top=1.0)
        if metric == 'ROC/AUC':
            axs[k].set_ylabel('ROC AUC')
        elif metric == 'f1':
            axs[k].set_ylabel('F1')
        else:
            axs[k].set_ylabel(metric)
        # axs[k].legend(bbox_to_anchor=(1.02, 1), loc='upper left')

        # Create legend handles for colors
        color_handles = [
            mpatches.Patch(color=colors[i], label=config[model]['lab']) for i, model in enumerate(models)
        ]

        # Create legend handles for marker shapes
        shape_handles = [
            mlines.Line2D([], [], color='black', marker='o', linestyle='None', markersize=10, label=modes[0]),
            mlines.Line2D([], [], color='black', marker='s', linestyle='None', markersize=10, label=modes[1]),
            mlines.Line2D([], [], color='black', marker='D', linestyle='None', markersize=10, label=modes[2]),
        ]

        # Add separate legends to the plot
        legend1 = axs[k].legend(handles=color_handles, title='Models', bbox_to_anchor=(1.0, 1), loc='upper left')
        legend2 = axs[k].legend(handles=shape_handles, title='Anomaly labels', bbox_to_anchor=(1.0, 0.5), loc='upper left')

        # Add the first legend back to the plot to ensure it shows up
        axs[k].add_artist(legend1)

    fig.subplots_adjust(right=0.5)  # Adjust the right padding to make space for the legend
    plt.tight_layout()
    if name:
        if metric == 'ROC/AUC':
            metric = 'rocauc'
        plt.savefig(f'./studies_results_lxplus/{name}_{metric}all.png', facecolor='w')
    plt.show()


In [20]:
def plot_best_scores_mean_std_err(results, datasets, models, metric='MCC', name=None, labels=None):
    fig, ax = plt.subplots(figsize=(20, 6))

    for j, model in enumerate(models):
        scores = {'mean': np.empty(0), 'std': np.empty(0)}
        for dataset in datasets:
            key = f'{model}_{dataset}'
            
            if key in results:
                scores['mean'] = np.append(scores['mean'], results[key]['mean'][metric].max())
                idx = np.where(results[key]['mean'][metric] == results[key]['mean'][metric].max())[0][0]
                scores['std'] = np.append(scores['std'], results[key]['std'][metric].iloc[idx])
            else:
                scores['mean'] = np.append(scores['mean'], 0)
                scores['std'] = np.append(scores['std'], 0)
        
        x_positions = np.arange(len(datasets)) + j * 0.1  # add offset for each model
        if labels:
            ax.errorbar(x_positions, scores['mean'], yerr=scores['std'], fmt='o', label=labels[j])  
        else:
            ax.errorbar(x_positions, scores['mean'], yerr=scores['std'], fmt='o', label=model) 
            
    ax.set_xticks(np.arange(len(datasets)) + 0.1 * (len(models) - 1) / 2)
    ax.set_xticklabels(datasets)
    ax.set_ylim(top=1.05)
    if metric == 'ROC/AUC':
        ax.set_ylabel('ROC AUC')
        ax.set_title(f'Best ROC AUC Scores')
        metric = 'rocauc'
    elif metric == 'f1':
        ax.set_ylabel('F1')
        ax.set_title(f'Best F1 Scores')
    else:
        ax.set_ylabel(metric)    
        ax.set_title(f'Best {metric} scores')
    ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left')

    plt.tight_layout()
    if name:
        plt.savefig(f'./studies_results_lxplus/{name}_{metric}best.png', facecolor='w')
    plt.show()

In [ ]:
data_plot = ['IEEECIS_new2.2', 'GECCO', 'SMD', 'SMAP_new', 'MSL_new', 'SWaT_1D', 'UCR'] 
models_plot = ['iTransformer1', 'iTransformer3', 'iTransformer2', 'iTransformer4', 'TranAD'] # 'MAD_GAN', 'OmniAnomaly', 'LSTM_AE', 'DAGMM', 'USAD']
lab = [config[m]['lab'] for m in models_plot]

name = None  # 'new'
plot_best_scores_mean_std_err(results_mean_std, data_plot, models_plot, metric='MCC', labels=lab, name=name)
plot_scores_mean_std_err(results_mean_std, modes, data_plot, models_plot, metric='MCC', labels=lab, name=name)
plot_scores_mean_std_err2(results_mean_std, modes, data_plot, models_plot, metric='MCC', labels=lab, name=name)

In [ ]:
def list_results(results, datasets, models, metric='MCC', labels=None, val='mean'):
    dict = {}
    for j, model in enumerate(models):
        mcc_scores = []
        for dataset in datasets:
            key = f'{model}_{dataset}'
            print(key)
            if key in results:
                mcc_scores.append(results[key][val][metric].max().round(3))
            else:
                mcc_scores.append(0)  # If no data, append 0

            dict[labels[j]] = mcc_scores

    return dict

In [37]:
def list_results2(results, datasets, models, metric='MCC', labels=None, val='mean'):
    df = pd.DataFrame()
    rows = []
    for j, model in enumerate(models):
        for i, mode in enumerate(modes):
            rows = pd.Series([results[f"{model}_{dataset}"][val][metric].iloc[i].round(3) if f"{model}_{dataset}" in results else 0 for dataset in datasets], index=datasets)
            # print(rows)
            complete_row = pd.Series({'model': labels[j], 'mode': mode})
            complete_row = pd.concat([complete_row, rows])
            df = pd.concat([df, pd.DataFrame(complete_row).T], ignore_index=True)

    
    print(df)
    return df


In [23]:
def list_results3(results, datasets, models, metric='MCC', labels=None):

    df = pd.DataFrame()
    rows = []
    for j, model in enumerate(models):
        for i, mode in enumerate(modes):
            i_datasets = [f'{dataset}_std' for dataset in datasets]
            rows = pd.Series([rf'{results[f"{model}_{dataset}"]["mean"][metric].iloc[i].round(3)} $\pm$ {results[f"{model}_{dataset}"]["std"][metric].iloc[i].round(3)}' if f"{model}_{dataset}" in results else 0 for dataset in datasets], index=datasets)
            # rows = pd.Series([results[f"{model}_{dataset}"]['mean'][metric].iloc[i].round(3) if f"{model}_{dataset}" in results else 0 for dataset in datasets], index=datasets)
            # rows2 = pd.Series([results[f"{model}_{dataset}"]['std'][metric].iloc[i].round(3) if f"{model}_{dataset}" in results else 0 for dataset in datasets], index=i_datasets)
            # interwoven = [p for pair in zip(rows, rows2) for p in pair]
            # print(interwoven)
            # interwoven_index = datasets
            # interwoven_index = [val for pair in zip(datasets, i_datasets) for val in pair]
            # rows = pd.Series(interwoven, index=interwoven_index)
            # print(rows)
            complete_row = pd.Series({'model': labels[j], 'mode': mode})
            complete_row = pd.concat([complete_row, rows])
            df = pd.concat([df, pd.DataFrame(complete_row).T], ignore_index=True)
    
    # print(df)
    return df

In [ ]:
val = 'mean'
metric = 'MCC'
namee = 'latent2_rep5_new'
data3 = ['IEEECIS_new2.2', 'GECCO', 'SMD', 'SMAP_new', 'MSL_new', 'SWaT_1D', 'UCR']
# ['ATLAS_TS', 'IEEECIS_new2.2', 'GECCO6', 'SMD', 'SMAP_new', 'MSL_new', 'SWaT', 'UCR'] #'ATLAS_TS',
models3 = ['iTransformer1', 'iTransformer2', 'TranAD'] # ['iTransformer1', 'iTransformer3', 'iTransformer2', 'iTransformer4', 'TranAD']

lab = [config[m]['lab'] for m in models3]

# dict_bestMCC = list_results(results_mean_std, data3, models3, metric, labels=lab, val=val)
# df_bestMCC2 = list_results2(results_mean_std, data3, models3, metric, labels=lab, val=val)
df_bestMCC3 = list_results3(results_mean_std, data3, models3, metric, labels=lab)
# df_bestMCC = pd.DataFrame(dict_bestMCC, index=data3).T
print(df_bestMCC3)
# print(df_bestMCC.values)
# metric = metric.replace(' ', '_')
# df_bestMCC.to_csv(f'studies_results_lxplus/{metric}_{namee}_{val}.csv')
# df_bestMCC2.to_csv(f'studies_results_lxplus/{metric}_{namee}_all.csv')
df_bestMCC3.to_csv(f'studies_datasets/{metric}_{namee}_all_latex.csv')